In [1]:
from dask.distributed import Client, progress
client = Client(n_workers=4, threads_per_worker=2, memory_limit='15GB')
client

Client Scheduler: tcp://127.0.0.1:51863 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 60.00 GB


In [2]:
import dask.dataframe as dd

In [3]:
da = dd.read_csv("Feb12_22.csv")
da.shape

(Delayed('int-ab240a3c-a06a-4230-b36b-972faab7508c'), 16)

In [4]:
da["model"] = 0  # random imp for A/B test
da["model"] = da.model.where(da.css == -1, 1)

In [5]:
%%time
da.groupby(["model"])["bidCnt", "uniquebidCnt", "publisher_revenue"].sum().compute()

CPU times: user 3.27 s, sys: 467 ms, total: 3.73 s
Wall time: 44.4 s


,bidCnt,uniquebidCnt,publisher_revenue
model,,,
0,54928187427,54090879806,1.237221e+08
1,18004641378,17812991404,1.634769e+07


In [6]:
%%time
da.groupby(["model", "hour"])["bidCnt", "uniquebidCnt", "publisher_revenue"].sum().compute().head()

CPU times: user 3.48 s, sys: 501 ms, total: 3.98 s
Wall time: 45.4 s


bidCnt  uniquebidCnt  publisher_revenue
model hour                                             
0     0     1907184664    1890990230       1.354381e+06
      1     1950266310    1933277412       1.356086e+06
      2     2279918241    2257340237       2.202128e+06
1     0      399467040     396747795       1.544616e+05
      1      409951990     407187637       1.608288e+05

# Memory
we can load all data to memory

In [7]:
da = client.persist(da)

In [8]:
%%time
da.groupby(["model", "hour"])["bidCnt", "uniquebidCnt", "publisher_revenue"].sum().compute().head()

CPU times: user 3.56 s, sys: 604 ms, total: 4.16 s
Wall time: 47.3 s


bidCnt  uniquebidCnt  publisher_revenue
model hour                                             
0     0     1907184664    1890990230       1.354381e+06
      1     1950266310    1933277412       1.356086e+06
      2     2279918241    2257340237       2.202128e+06
1     0      399467040     396747795       1.544616e+05
      1      409951990     407187637       1.608288e+05

In [9]:
%%time
da.groupby(["model", "hour"])["bidCnt", "uniquebidCnt", "publisher_revenue"].sum().compute().head()

CPU times: user 467 ms, sys: 64.4 ms, total: 531 ms
Wall time: 2.26 s


bidCnt  uniquebidCnt  publisher_revenue
model hour                                             
0     0     1907184664    1890990230       1.354381e+06
      1     1950266310    1933277412       1.356086e+06
      2     2279918241    2257340237       2.202128e+06
1     0      399467040     396747795       1.544616e+05
      1      409951990     407187637       1.608288e+05

## convert dask to pandas

In [11]:
df = da.compute()
type(df)

pandas.core.frame.DataFrame

# parallel function

In [10]:
def twotimes(x):
    return 2*x

In [18]:
%%time
df['bidCnt'].apply(lambda x: twotimes(x)).head()

CPU times: user 33.8 s, sys: 2.61 s, total: 36.4 s
Wall time: 36.3 s


0    2
1    2
2    2
3    2
4    4
Name: bidCnt, dtype: int64

In [19]:
from dask import delayed, compute

In [24]:
result = [delayed(twotimes)(x) for x in df[:1000]['bidCnt'].values]

In [25]:
%%time
out = compute(result)

CPU times: user 599 ms, sys: 115 ms, total: 713 ms
Wall time: 5.44 s


# it is possible to scatter data to minimize comunication time
[modeldata_scatered] = client.scatter([modeldata], broadcast=True)